<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# HubSpot - Chat about HubSpot contact activities
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/HubSpot/HubSpot_Get_activities_from_contact.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Get+activities+from+contact:+Error+short+description">Bug report</a>

**Tags:** #hubspot #contact #activity #notes #emails #communications #meetings #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-08-21 (Created: 2023-08-21)

**Description:** This notebook demonstrates how to retrieve all activities from a contact URL in HuSpot in use it in Naas Chat.

**References:**
- [HubSpot API - Contacts](https://developers.hubspot.com/docs/api/crm/contacts)
- [HubSpot API - Associations v4](https://developers.hubspot.com/docs/api/crm/associations)
- [HubSpot API - Communications](https://developers.hubspot.com/docs/api/crm/communications)

## Input

### Import libraries

In [ ]:
import requests
import naas
from naas_drivers import hubspot
import pandas as pd
import json
try:
    import tiktoken
except:
    !pip install tiktoken --user
    import tiktoken
pd.set_option('display.max_colwidth', None)

### Setup variables
**Mandatory**

[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `contact_url`: This variable stores the HubSpot contact URL.

**Optional**
- `plugin_name`: It represents the name of the plugin.
- `plugin_model`: It specifies the model to be used by the plugin.
- `plugin_temperature`: It determines the creativity level of the generated content, with higher values resulting in more diverse outputs.
- `plugin_max_tokens`: It specifies the number of maximum tokens to be used by the plugin.
- `plugin_file_name`: This variable contains the name of the plugin file that will analyze the posts.

In [ ]:
# Mandatory
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
contact_url = "https://app.hubspot.com/contacts/2474088/record/0-1/315551" # "https://app.hubspot.com/contacts/xxxxx/record/0-1/xxxxx"

# Optional
contact_properties = ["hs_object_id", "firstname", "lastname", 'email', 'linkedinbio', 'jobtitle']
associations = ["notes", "emails", "meetings", "communications"]
plugin_name = "Sales Agent"
plugin_model = "gpt-3.5-turbo-16k"
plugin_temperature = 0
plugin_max_tokens = 16384
plugin_file_name = "posts_analyzer_plugin.json"

In [ ]:
# Parameters
try:
    contact_url = body.get("hubspot_profile_url")
except:
    pass

## Model

#### Engineer system prompt
We used Playground to refined it: https://platform.openai.com/playground?mode=chat&model=gpt-4

In [ ]:
system_prompt = f"""Act as a Sales Agent who has access to the Contact data from the CRM. 
Your role is to get data from a Hubspot Contact and define the next steps to be done to close a deal.
You can use the function /hubspot_profile_url and copy/paste the HubSpot URL to get information from any contact in your CRM.
"""

#### Check tokens count

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

system_prompt_tokens = num_tokens_from_string(system_prompt, "cl100k_base")
if system_prompt_tokens > plugin_max_tokens * 0.8:
    print("⚠️ Be carefull, your system prompt looks too big. Tokens:", system_prompt_tokens)
else:
    print("✅ System prompt tokens count OK:", system_prompt_tokens, f'(limit: 20% -> {int(plugin_max_tokens * 0.2)})')

### Get contact ID from URL

In [ ]:
def get_contact_ID_from_URL(url):
    # Init
    uid = url
    
    # Check if URL is valid
    if not url.startswith("https://app.hubspot.com/contacts/"):
        raise BaseException("HubSpot URL Invalid! It must start by https://app.hubspot.com/contacts/")
    
    # Split URL to get ID
    if "/record/0-1/" in url:
        uid = url.split("/record/0-1/")[-1].split("/")[0]
    return uid

contact_id = get_contact_ID_from_URL(contact_url)
print("Contact ID:", contact_id)

### Get contact details

In [ ]:
def retrieve_object_details(
    token,
    object_id,
    object_type,
    properties=None,
):
    # Init
    data = []
    params = {
        "archived": "false"
    }
    
    # Requests
    if properties:
        params["properties"] = properties
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.hubapi.com/crm/v3/objects/{object_type}/{object_id}"
    
    # Response
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        data = res.json().get("properties")
    else:
        print(res.text)
    return pd.DataFrame([data])

def create_activity_df(
    token,
    object_id,
    activity,
    properties_dict=None,
):
    # Init
    properties = [x for x in properties_dict]
    
    # List activities
    df = retrieve_object_details(
        token,
        object_id,
        activity,
        properties
    )
    if len(df) > 0:
        df = df[properties]
        
    if len(df) > 0:
        df = df.rename(columns=properties_dict)
        if 'activity_type' not in df:
            df.insert(loc=1, column="activity_type", value=activity.upper())
    
    return df.reset_index(drop=True)

def get_contact_details(
    hs_access_token,
    contact_id,
    properties,
    associations
):
    # Init
    message = "CONTACT:\n"
    df = pd.DataFrame()
    
    # Get contact
    contact = hubspot.connect(hs_access_token).contacts.get(
        contact_id,
        hs_properties=properties,
        hs_associations=associations
    )
    
    # Get contact properties
    contact_properties = contact.get("properties")
    for p in properties:
        message = f"{message}- {p}: {contact_properties.get(p)}\n"
    
    # Get contact associations
    contact_associations = contact.get("associations")
    for a in contact_associations:
        results = contact_associations.get(a).get("results")
        for r in results:
            if a == "communications":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_lastmodifieddate": "activity_date",
                    "hs_communication_channel_type": "activity_type",
                    "hs_body_preview": "activity_content"
                }
            elif a == "meetings":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_lastmodifieddate": "activity_date",
                    "hs_meeting_title": "activity_content"
                }
            else:
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_lastmodifieddate": "activity_date",
                    "hs_body_preview": "activity_content"
                }
            association_id = r.get("id")
            
            # Create activity df
            tmp_df = create_activity_df(
                hs_access_token,
                association_id,
                a,
                properties_dict
            )
            df = pd.concat([df, tmp_df])
    
    # Cleaning df
    if len(df) > 0:
        # Exclude empty or None value
        df = df[~(df["activity_content"].astype(str).isin(["None"]))]

        # Format date
        df["activity_date"] = pd.to_datetime(df["activity_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
        df = df.sort_values(by="activity_date").reset_index(drop=True)
            
    # Create activity message
    message = f"{message}\nACTIVITIES:\n"
    for row in df.itertuples():
        activity_date = row.activity_date
        activity_type = row.activity_type
        activity_content = row.activity_content.replace("\xa0\u200c", "")
        message = f"{message}-{activity_date}: {activity_type} - {activity_content}\n"
    return message, df.reset_index(drop=True)

prompt_message, df_activity = get_contact_details(
    hs_access_token,
    contact_id,
    contact_properties,
    associations
)
print(prompt_message)

## Output

### Save DataFrame to CSV

In [ ]:
df_activity.to_csv(f"contact_activity_{contact_id}.csv", index=False)

### Add webhook

In [ ]:
webhook_url = naas.webhook.add(params={"inline": True})

# to de-schedule this notebook, simply run the following command:
# naas.scheduler.delete()

### Create webhook response

In [ ]:
response_json = {
    "status": "OK",
    "message": prompt_message,
}
naas.webhook.respond_json(response_json)

### Generate plugin
Plugin must be a JSON file with mandatory keys name, model, temperature, max_tokens and prompt

In [ ]:
# Create json
plugin = {
    "name": plugin_name,
    "model": plugin_model,
    "temperature": plugin_temperature,
    "max_tokens": plugin_max_tokens,
    "prompt": system_prompt,
    "commands": [
        { 
            "name": "HubSpot_Create_Contact_from_LinkedIn_Profile",
            "action": {
                "request_type": "POST",
                "url": webhook_url,
                "payload": {
                    "hubspot_profile_url": {
                        "type": "str",
                        "description": "HubSpot profile url that you want to fetch. Ex: https://app.hubspot.com/contacts/2474088/record/0-1/308551/view/1",
                        "default": ""
                    },
                }
            }
        }
    ]
}

# Save dict to JSON file
with open(plugin_file_name, "w") as f:
    json.dump(plugin, f)
print("💾 Plugin successfully saved:", plugin_file_name)

### Create asset
You can now use in your Naas Chat by copy/pasting the URL after the command `/use `

In [ ]:
naas.asset.add(plugin_file_name, params={"inline": True})